In [1]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install quandl

In [2]:
import pandas as pd
import quandl

df = quandl.get("WIKI/GOOGL")

print(df.head)

<bound method NDFrame.head of                Open     High      Low     Close      Volume  Ex-Dividend  \
Date                                                                       
2004-08-19   100.01   104.06    95.96   100.335  44659000.0          0.0   
2004-08-20   101.01   109.08   100.50   108.310  22834300.0          0.0   
2004-08-23   110.76   113.48   109.05   109.400  18256100.0          0.0   
2004-08-24   111.24   111.60   103.57   104.870  15247300.0          0.0   
2004-08-25   104.76   108.00   103.88   106.000   9188600.0          0.0   
...             ...      ...      ...       ...         ...          ...   
2018-03-21  1092.57  1108.70  1087.21  1094.000   1990515.0          0.0   
2018-03-22  1080.01  1083.92  1049.64  1053.150   3418154.0          0.0   
2018-03-23  1051.37  1066.78  1024.87  1026.550   2413517.0          0.0   
2018-03-26  1050.60  1059.27  1010.58  1054.090   3272409.0          0.0   
2018-03-27  1063.90  1064.54   997.62  1006.940   2940957.

In [4]:
#Thus, let's go ahead and pair down our original dataframe a bit:
df = df[['Adj. Open',  'Adj. High',  'Adj. Low',  'Adj. Close', 'Adj. Volume']]

# Let's go ahead and transform our data next:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Low']) / df['Adj. Close'] * 100.0

#Next, we'll do daily percent change:
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.


#Now we will define a new dataframe as:

df = df[['Adj. Close', 'HL_PCT', 'PCT_change', 'Adj. Volume']]
print(df.head())

            Adj. Close    HL_PCT  PCT_change  Adj. Volume
Date                                                     
2004-08-19   50.322842  8.072956    0.324968   44659000.0
2004-08-20   54.322689  7.921706    7.227007   22834300.0
2004-08-23   54.869377  4.049360   -1.227880   18256100.0
2004-08-24   52.597363  7.657099   -5.726357   15247300.0
2004-08-25   53.164113  3.886792    1.183658    9188600.0


In [5]:
import quandl, math
import numpy as np
import pandas as pd
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression

In [6]:
#math.ceil(x)
#Return the ceiling of x as a float,
#the smallest integer value greater than or equal to x.

#Forcost, fill missing data and return ceiling as float, smallest integer values
#greater than or equal to x
#In our case, we've decided the features are a bunch of the current values, and the label shall be the price, in the future, where the future is 1% of the entire length of the dataset out. We'll assume all current columns are our features,
#so we'll add a new column with a simple pandas operation:
#We're saying we want to forecast out 1% of the entire length of the dataset
forecast_col = 'Adj. Close'
df.fillna(value=-99999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df)))


In [7]:
#In our case, we've decided the features are a bunch of the current values,
#and the label shall be the price, in the future, where the future is 1% of the entire length of the dataset out.
#We'll assume all current columns are our features, so we'll add a new column with a simple pandas operation:

df['label'] = df[forecast_col].shift(-forecast_out)


#Regression - Training and Testing
#We'll then drop any still NaN information from the dataframe:







#df.dropna(inplace=True)       
#X = np.array(df.drop(['label'], 1))
#y = np.array(df['label'])
#X = preprocessing.scale(X)
#y = np.array(df['label'])
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#clf = svm.SVR()
#clf = LinearRegression()
#clf.fit(X_train, y_train)
#confidence = clf.score(X_test, y_test)
#print(confidence)
#clf = LinearRegression()
#clf = LinearRegression(n_jobs=-1)
#for k in ['linear','poly','rbf','sigmoid']:
#    clf = svm.SVR(kernel=k)
#    clf.fit(X_train, y_train)
#   confidence = clf.score(X_test, y_test)
#    print(k,confidence)
#Predict
X = np.array(df.drop(['label'], 1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:]
X = X[:-forecast_out]
df.dropna(inplace=True)
y = np.array(df['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train, y_train)
confidence = clf.score(X_test, y_test)
print(confidence)

0.974995394910679


In [8]:
forecast_set = clf.predict(X_lately)
print(forecast_set, confidence, forecast_out)

[1098.20482897 1069.04633027 1014.3149237  1055.49336593 1070.53942255
 1071.43311416 1089.23024356 1107.55927465 1112.59561664 1120.11499103
 1129.73261735 1126.46936866 1145.51145223 1161.85498054 1132.51210176
 1119.29724581 1083.4334524  1099.64921741 1111.21550652 1118.79845976
 1132.12611938 1147.51495925 1178.00999621 1183.19483413 1153.1758297
 1166.30908265 1167.08832726 1149.797062   1114.57004102 1111.76429654
 1110.51939342 1066.30531357 1038.91454048 1066.78576007 1014.62236609] 0.974995394910679 35


In [9]:
#visualizing this information
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
df['Forecast'] = np.nan


last_date = df.iloc[-1].name
last_unix = last_date.timestamp()
one_day = 86400
next_unix = last_unix + one_day
for i in forecast_set:
    next_date = datetime.datetime.fromtimestamp(next_unix)
    next_unix += 86400
    df.loc[next_date] = [np.nan for _ in range(len(df.columns)-1)]+[i]

df['Adj. Close'].plot()
df['Forecast'].plot()
plt.legend(loc=4)
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

<Figure size 640x480 with 1 Axes>